In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import cv2
import torch
import random
from glob import glob
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from torch.utils.data import Dataset

seed = 50
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.enabled = False

%matplotlib inline

In [3]:
path = '/content/drive/MyDrive/data/programmers_ai/mlops/COVID_19_XRAY_2/'

In [4]:
train = pd.read_csv(os.path.join(path, 'train/labels.csv'))
train.head

<bound method NDFrame.head of             filename   label
0      image_001.png   covid
1      image_002.png   covid
2      image_003.png  normal
3      image_004.png  normal
4      image_005.png  normal
...              ...     ...
1995  image_1996.png   covid
1996  image_1997.png  normal
1997  image_1998.png   covid
1998  image_1999.png  normal
1999  image_2000.png  normal

[2000 rows x 2 columns]>

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   filename  2000 non-null   object
 1   label     2000 non-null   object
dtypes: object(2)
memory usage: 31.4+ KB


In [6]:
label_encoding = {'normal': 0, 'covid': 1}
train['label'] = train['label'].apply(lambda x: label_encoding[x])

In [7]:
train, valid = train_test_split(train, test_size=0.1, stratify=train['label'], random_state=34)

In [8]:
train.shape, valid.shape

((1800, 2), (200, 2))

In [9]:
train.head()

filename  label
1938  image_1939.png      0
444    image_445.png      1
1843  image_1844.png      1
1666  image_1667.png      0
106    image_107.png      0

In [10]:
train.iloc[0, 0]

'image_1939.png'

In [11]:
image = cv2.imread(os.path.join(path, 'train/image_1964.png'))
image.shape

(299, 299, 3)

In [12]:
plt.imshow(image)

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

device

device(type='cuda')

## 데이터셋 클래스 정의

In [14]:
class ImageDataset(Dataset):
    def __init__(self, df, img_dir='./', transform=None, is_test=False):
        super().__init__()
        self.df = df
        self.img_dir = img_dir
        self.transform = transform
        self.is_test = is_test
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        img_id = self.df.iloc[idx, 0]
        img_path = self.img_dir + img_id
        image = cv2.imread(img_path)
        
        if self.transform:
            image = self.transform(image)
        
        if self.is_test:
            return image
        else:
            label = self.df.iloc[idx, 1]
            return image, label

## 이미지 변환기 정의

In [15]:
from torchvision import transforms

# 훈련 데이터용 변환기
transform_train = transforms.Compose([
                          transforms.ToPILImage(),
                          transforms.Resize((224, 224)),      # 이미지 크기 조정 
                          transforms.CenterCrop(180),         # 중앙 이미지 확대
                          transforms.RandomHorizontalFlip(0.5), # 좌우 대칭
                          transforms.RandomVerticalFlip(0.2), # 상하 대칭
                          transforms.RandomRotation(20),      # 이미지 회전
                          transforms.ToTensor(),              # 텐서 객체로 변환
                          transforms.Normalize((0.485, 0.456, 0.406), 
                                               (0.229, 0.224, 0.225))]) # 정규화

# 테스트 데이터용 변환기
transform_test = transforms.Compose([
                          transforms.ToPILImage(),
                          transforms.Resize((224, 224)),
                          transforms.CenterCrop(180),
                          transforms.ToTensor(),
                          transforms.Normalize((0.485, 0.456, 0.406), 
                                               (0.229, 0.224, 0.225))])

## 데이터셋 및 데이터 로더 생성

In [16]:
dataset_train = ImageDataset(train, img_dir=os.path.join(path,'train/'), transform=transform_train)
dataset_valid = ImageDataset(valid, img_dir=os.path.join(path, 'train/'), transform=transform_test)

In [17]:
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

g = torch.Generator()
g.manual_seed(0)

In [18]:
from torch.utils.data import DataLoader

batch_size = 20

loader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, worker_init_fn=seed_worker, generator=g, num_workers=2)
loader_valid = DataLoader(dataset_valid, batch_size=batch_size, shuffle=False, worker_init_fn=seed_worker, generator=g, num_workers=2)

In [19]:
!pip install efficientnet-pytorch==0.7.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16444 sha256=1682d6977d517b622f949897c19399820f0c17f418831d74c3cfdfacba313a39
  Stored in directory: /root/.cache/pip/wheels/29/16/24/752e89d88d333af39a288421e64d613b5f652918e39ef1f8e3
Successfully built efficientnet-pytorch


In [20]:
from efficientnet_pytorch import EfficientNet

In [21]:
model = EfficientNet.from_pretrained('efficientnet-b7', num_classes=2)
model = model.to(device)

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b7-dcc49843.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b7-dcc49843.pth
100%|██████████| 254M/254M [00:07<00:00, 34.0MB/s]


Loaded pretrained weights for efficientnet-b7


In [22]:
print(model)

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 64, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d((0, 1, 0, 1))
  )
  (_bn0): BatchNorm2d(64, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        64, 64, kernel_size=(3, 3), stride=[1, 1], groups=64, bias=False
        (static_padding): ZeroPad2d((1, 1, 1, 1))
      )
      (_bn1): BatchNorm2d(64, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        64, 16, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        16, 64, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False
  

In [23]:
import torch.nn as nn

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.00006, weight_decay=0.0001)

In [24]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 51.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 113.5 MB/s eta 0:00:00


In [25]:
from transformers import get_cosine_schedule_with_warmup

epochs = 30 # 총 에폭

# 스케줄러 생성
scheduler = get_cosine_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=len(loader_train)*3, 
                                            num_training_steps=len(loader_train)*epochs)

In [26]:
from sklearn.metrics import accuracy_score # 정확도 계산 함수
from sklearn.metrics import recall_score   # 재현율 계산 함수
from sklearn.metrics import f1_score 
from tqdm.notebook import tqdm

for epoch in range(epochs):
    # 훈련
    model.train() # 모델을 훈련 상태로 설정
    epoch_train_loss = 0 # 에폭별 손실값 초기화(훈련 데이터용)
    
    for images, labels in tqdm(loader_train):
      images = images.to(device)
      labels = labels.to(device)
        
      optimizer.zero_grad() # 옵티마이저 내 기울기 초기화
      outputs = model(images) # 순전파 : 이미지 데이터를 신경망 모델의 입력값으로 사용해 출력값 계산
      loss = criterion(outputs, labels) # 손실 함수를 활용해 outputs와 labels의 손실값 계산
      epoch_train_loss += loss.item() # 현제 배치에서 손실 추가(훈련 데이터용)
      loss.backward() # 역전파 수행
      optimizer.step() # 가중치 갱신
      scheduler.step() # 스케줄러 학습률 갱신
    print(f'에폭 {epoch + 1}/{epochs} - 훈련 데이터 손실값: {epoch_train_loss / len(loader_train): .4f}')
    
    # 검증
    model.eval() # 모델을 평가 상태로 설정
    epoch_valid_loss = 0 # 에폭별 손실값 초기화(검증 데이터 용)
    preds_list = [] # 예측 확률값 저장용 리스트 초기화
    true_list = [] # 실제 타깃값 저장용 리스트 초기화
    
    with torch.no_grad(): # 기울기 계산 비활성화
        # 미니배치 단위로 검증
        for images, labels in loader_valid:
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            epoch_valid_loss += loss.item()
            
            preds = torch.max(outputs.cpu(), dim=1)[1].numpy() 
            true = labels.cpu().numpy() 

            preds_list.extend(preds)
            true_list.extend(true)
    # 정확도, 재현율, F1 점수 계산
    val_accuracy = accuracy_score(true_list, preds_list)
    val_recall = recall_score(true_list, preds_list)
    val_f1_score = f1_score(true_list, preds_list)

    # 검증 데이터 손실값 및 정확도, 재현율, F1점수 출력
    print(f'\t검증 데이터 손실값 : {epoch_valid_loss/len(loader_valid):.4f}')
    print(f'\t정확도 : {val_accuracy:.4f} / 재현율 : {val_recall:.4f} / F1 점수 : {val_f1_score:.4f}')

  0%|          | 0/90 [00:00<?, ?it/s]

에폭 1/30 - 훈련 데이터 손실값:  0.6762
	검증 데이터 손실값 : 0.6755
	정확도 : 0.5450 / 재현율 : 0.1200 / F1 점수 : 0.2087


  0%|          | 0/90 [00:00<?, ?it/s]

에폭 2/30 - 훈련 데이터 손실값:  0.5212
	검증 데이터 손실값 : 0.5289
	정확도 : 0.7200 / 재현율 : 0.4400 / F1 점수 : 0.6111


  0%|          | 0/90 [00:00<?, ?it/s]

...

에폭 28/30 - 훈련 데이터 손실값:  0.0129
	검증 데이터 손실값 : 0.0899
	정확도 : 0.9650 / 재현율 : 0.9700 / F1 점수 : 0.9652


  0%|          | 0/90 [00:00<?, ?it/s]

에폭 29/30 - 훈련 데이터 손실값:  0.0195
	검증 데이터 손실값 : 0.0899
	정확도 : 0.9650 / 재현율 : 0.9700 / F1 점수 : 0.9652


  0%|          | 0/90 [00:00<?, ?it/s]

에폭 30/30 - 훈련 데이터 손실값:  0.0147
	검증 데이터 손실값 : 0.0901
	정확도 : 0.9650 / 재현율 : 0.9700 / F1 점수 : 0.9652


## test 및 제출

In [27]:
submission = pd.read_csv(os.path.join(path,'submission.csv'))
submission.head()

filename  label
0  image_001.png    NaN
1  image_002.png    NaN
2  image_003.png    NaN
3  image_004.png    NaN
4  image_005.png    NaN

In [28]:
dataset_test = ImageDataset(submission, img_dir=os.path.join(path,'test/'), transform=transform_test, is_test=True)
loader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=False, worker_init_fn=seed_worker, generator=g, num_workers=2)

# TTA용 데이터셋 및 데이터 로더
dataset_TTA = ImageDataset(submission, img_dir=os.path.join(path,'test/'), transform=transform_train, is_test=True)
loader_TTA = DataLoader(dataset_TTA, batch_size=batch_size, shuffle=False, worker_init_fn=seed_worker,generator=g, num_workers=2)

In [29]:
model.eval() # 모델을 평가 상태로 설정 

preds_test = [] # 예측값 저장용 배열 초기화

with torch.no_grad():
    for i, images in enumerate(loader_test):
        images = images.to(device)
        outputs = model(images)
        # 타깃 예측 확률
        preds = torch.max(outputs.cpu(), dim=1)[1].numpy()
        preds_test.extend(preds)

In [30]:
submission_test = submission.copy()

In [31]:
info = {0: 'normal', 1:'covid'}

preds_test =list(map(lambda x: info[x], preds_test))

In [32]:
preds_test[:5]

['covid', 'covid', 'normal', 'covid', 'normal']

In [33]:
submission_test['label'] = preds_test

In [34]:
model.eval()

num_TTA = 7 # TTA 횟수
preds_tta = np.zeros(len(submission))

for i in range(num_TTA):
  with torch.no_grad():
    for i, images in enumerate(loader_TTA):
        images = images.to(device)
        outputs = model(images)
        preds = torch.max(outputs.cpu(), dim=1)[1].numpy()
        preds_tta[i*batch_size: (i+1)*batch_size] += preds

In [35]:
preds_tta /= num_TTA
preds_tta = np.round(preds_tta,0)

In [36]:
info = {0: 'normal', 1:'covid'}

preds_list =list(map(lambda x: info[x], list(preds_tta)))
preds_list[:10]

['covid',
 'covid',
 'normal',
 'covid',
 'normal',
 'normal',
 'covid',
 'covid',
 'normal',
 'covid']

In [37]:
submission['label'] = preds_list

In [38]:
submission.to_csv('submission_ttl.csv',index=False)
submission_test.to_csv('submission_test.csv', index=False)